# Model Experimenting
This notebook will work as an experiment on how well different ML models do on historical data for different stocks.

## Importing

In [7]:
from typing import Union
import numpy as np
import sys
import datetime as dt
import pandas as pd

from pathlib import Path
sys.path.append(str(Path("..").resolve()))

from live_trader.ml_model import ML_Pipeline
from live_trader.ml_model.ml_strategies import AI_strategy, attention_bilstm_strategy

In [11]:
# Tensorflow
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Dropout, Bidirectional,
    Attention, LayerNormalization, Add, GlobalAveragePooling1D, Conv1D
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping

## Testing our models that are already made

### Basic LSTM

In [ ]:
side, _ = await AI_strategy("GOOG")
print(f"GOOG: {side}")

Epoch 1/20
63/63 - 2s - 25ms/step - loss: 0.6996 - roc_auc: 0.4737 - val_loss: 0.6905 - val_roc_auc: 0.4577
Epoch 2/20
63/63 - 0s - 3ms/step - loss: 0.6908 - roc_auc: 0.5319 - val_loss: 0.6862 - val_roc_auc: 0.4514
Epoch 3/20
63/63 - 0s - 3ms/step - loss: 0.6913 - roc_auc: 0.5224 - val_loss: 0.6913 - val_roc_auc: 0.4620
Epoch 4/20
63/63 - 0s - 3ms/step - loss: 0.6887 - roc_auc: 0.5316 - val_loss: 0.6864 - val_roc_auc: 0.4627
Epoch 5/20
63/63 - 0s - 3ms/step - loss: 0.6912 - roc_auc: 0.5221 - val_loss: 0.6911 - val_roc_auc: 0.4774
Epoch 6/20
63/63 - 0s - 3ms/step - loss: 0.6899 - roc_auc: 0.5331 - val_loss: 0.6865 - val_roc_auc: 0.4806
Epoch 7/20
63/63 - 0s - 3ms/step - loss: 0.6868 - roc_auc: 0.5478 - val_loss: 0.6856 - val_roc_auc: 0.4829
Epoch 8/20
63/63 - 0s - 3ms/step - loss: 0.6867 - roc_auc: 0.5475 - val_loss: 0.6893 - val_roc_auc: 0.4799
Epoch 9/20
63/63 - 0s - 3ms/step - loss: 0.6879 - roc_auc: 0.5377 - val_loss: 0.6845 - val_roc_auc: 0.4869
Epoch 10/20
63/63 - 0s - 4ms/step - 

In [4]:
side, _ = await AI_strategy("AAPL")
print(f"AAPL: {side}")

Epoch 1/20
63/63 - 2s - 24ms/step - loss: 0.6977 - roc_auc: 0.4982 - val_loss: 0.7032 - val_roc_auc: 0.4854
Epoch 2/20
63/63 - 0s - 4ms/step - loss: 0.6964 - roc_auc: 0.5114 - val_loss: 0.7020 - val_roc_auc: 0.5193
Epoch 3/20
63/63 - 0s - 3ms/step - loss: 0.6900 - roc_auc: 0.5407 - val_loss: 0.7107 - val_roc_auc: 0.4703
Epoch 4/20
63/63 - 0s - 3ms/step - loss: 0.6938 - roc_auc: 0.5271 - val_loss: 0.7048 - val_roc_auc: 0.4771
Epoch 5/20
63/63 - 0s - 3ms/step - loss: 0.6893 - roc_auc: 0.5391 - val_loss: 0.7176 - val_roc_auc: 0.4602
Epoch 6/20
63/63 - 0s - 3ms/step - loss: 0.6896 - roc_auc: 0.5455 - val_loss: 0.7156 - val_roc_auc: 0.4753
Epoch 7/20
63/63 - 0s - 3ms/step - loss: 0.6907 - roc_auc: 0.5485 - val_loss: 0.7214 - val_roc_auc: 0.4809
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
AAPL: SideSignal.BUY


In [5]:
side, _ = await AI_strategy("MCFT")
print(f"MCFT: {side}")

Epoch 1/20
63/63 - 2s - 24ms/step - loss: 0.7014 - roc_auc: 0.5083 - val_loss: 0.6960 - val_roc_auc: 0.4777
Epoch 2/20
63/63 - 0s - 4ms/step - loss: 0.6912 - roc_auc: 0.5537 - val_loss: 0.6998 - val_roc_auc: 0.4568
Epoch 3/20
63/63 - 0s - 3ms/step - loss: 0.6872 - roc_auc: 0.5603 - val_loss: 0.7010 - val_roc_auc: 0.4638
Epoch 4/20
63/63 - 0s - 3ms/step - loss: 0.6806 - roc_auc: 0.5898 - val_loss: 0.7051 - val_roc_auc: 0.4534
Epoch 5/20
63/63 - 0s - 3ms/step - loss: 0.6825 - roc_auc: 0.5750 - val_loss: 0.7072 - val_roc_auc: 0.4450
Epoch 6/20
63/63 - 0s - 3ms/step - loss: 0.6821 - roc_auc: 0.5823 - val_loss: 0.7085 - val_roc_auc: 0.4491
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
MCFT: SideSignal.BUY


### attention bilstm

In [8]:
side, _ = await attention_bilstm_strategy("GOOG")
print(f"GOOG: {side}")

Epoch 1/20
63/63 - 3s - 46ms/step - loss: 0.7428 - roc_auc: 0.5119 - val_loss: 0.6878 - val_roc_auc: 0.4823
Epoch 2/20
63/63 - 0s - 6ms/step - loss: 0.7196 - roc_auc: 0.5172 - val_loss: 0.7657 - val_roc_auc: 0.5266
Epoch 3/20
63/63 - 0s - 5ms/step - loss: 0.7045 - roc_auc: 0.5278 - val_loss: 0.6585 - val_roc_auc: 0.5280
Epoch 4/20
63/63 - 0s - 5ms/step - loss: 0.7058 - roc_auc: 0.5125 - val_loss: 0.6771 - val_roc_auc: 0.5436
Epoch 5/20
63/63 - 2s - 29ms/step - loss: 0.6881 - roc_auc: 0.5523 - val_loss: 0.6699 - val_roc_auc: 0.5040
Epoch 6/20
63/63 - 0s - 5ms/step - loss: 0.6939 - roc_auc: 0.5328 - val_loss: 0.6761 - val_roc_auc: 0.5429
Epoch 7/20
63/63 - 0s - 5ms/step - loss: 0.6968 - roc_auc: 0.5147 - val_loss: 0.6730 - val_roc_auc: 0.5479
Epoch 8/20
63/63 - 0s - 5ms/step - loss: 0.6903 - roc_auc: 0.5291 - val_loss: 0.7092 - val_roc_auc: 0.5383


/home/milad/repositories/Live-Trading-Platform/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/milad/repositories/Live-Trading-Platform/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/milad/repositories/Live-Trading-Platform/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control thi

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
GOOG: SideSignal.BUY


In [9]:
side, _ = await attention_bilstm_strategy("AAPL")
print(f"AAPL: {side}")

Epoch 1/20
63/63 - 3s - 47ms/step - loss: 0.7684 - roc_auc: 0.5264 - val_loss: 0.7679 - val_roc_auc: 0.4356
Epoch 2/20
63/63 - 0s - 6ms/step - loss: 0.7191 - roc_auc: 0.5211 - val_loss: 0.6851 - val_roc_auc: 0.5550
Epoch 3/20
63/63 - 0s - 5ms/step - loss: 0.7149 - roc_auc: 0.5187 - val_loss: 0.7715 - val_roc_auc: 0.4205
Epoch 4/20
63/63 - 0s - 6ms/step - loss: 0.7015 - roc_auc: 0.5472 - val_loss: 0.7245 - val_roc_auc: 0.4609
Epoch 5/20
63/63 - 0s - 5ms/step - loss: 0.6985 - roc_auc: 0.5370 - val_loss: 0.7247 - val_roc_auc: 0.4172
Epoch 6/20
63/63 - 0s - 5ms/step - loss: 0.6964 - roc_auc: 0.5539 - val_loss: 0.7537 - val_roc_auc: 0.4132
Epoch 7/20
63/63 - 0s - 5ms/step - loss: 0.6817 - roc_auc: 0.5813 - val_loss: 0.7067 - val_roc_auc: 0.4672
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
AAPL: SideSignal.HOLD


In [10]:
side, _ = await attention_bilstm_strategy("MCFT")
print(f"MCFT: {side}")

Epoch 1/20
63/63 - 3s - 51ms/step - loss: 0.7649 - roc_auc: 0.5191 - val_loss: 0.7598 - val_roc_auc: 0.4743
Epoch 2/20
63/63 - 0s - 6ms/step - loss: 0.7361 - roc_auc: 0.5336 - val_loss: 0.7169 - val_roc_auc: 0.4638
Epoch 3/20
63/63 - 0s - 5ms/step - loss: 0.7129 - roc_auc: 0.5286 - val_loss: 0.7228 - val_roc_auc: 0.4890
Epoch 4/20
63/63 - 0s - 5ms/step - loss: 0.7027 - roc_auc: 0.5520 - val_loss: 0.7556 - val_roc_auc: 0.4203
Epoch 5/20
63/63 - 0s - 5ms/step - loss: 0.7046 - roc_auc: 0.5362 - val_loss: 0.7387 - val_roc_auc: 0.4193
Epoch 6/20
63/63 - 0s - 5ms/step - loss: 0.6872 - roc_auc: 0.5659 - val_loss: 0.7635 - val_roc_auc: 0.4630
Epoch 7/20
63/63 - 0s - 5ms/step - loss: 0.6920 - roc_auc: 0.5528 - val_loss: 0.7281 - val_roc_auc: 0.4324
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
MCFT: SideSignal.BUY


Both attention_bilstm and basic_lstm are not good models. Therefore, we will try out other models as well.

## Modelling

### Testing out Temporal Convolutional Network (TCN-lite)
- learns local temporal patterns
- no reccurence (more stable)
- strong inductibe bias for noisy sequences
- much easier to train

In [12]:
def build_tcn_lite(X_train_seq: Union[np.ndarray, list]) -> Model:
    """
    Builds a lightweight Temporal Convolutional Network (TCN-style)
    for noisy financial time series classification.

    Designed to be robust to non-stationarity and overfitting.

    Args:
        X_train_seq (array-like):
            Training sequences of shape (n_samples, time_steps, n_features)

    Returns:
        Compiled Keras Model
    """
    n_features = X_train_seq.shape[2]

    inputs = Input(shape=(None, n_features))

    x = Conv1D(
        filters=32,
        kernel_size=3,
        padding="causal",
        activation="relu"
    )(inputs)
    x = LayerNormalization()(x)
    x = Dropout(0.3)(x)

    x = Conv1D(
        filters=16,
        kernel_size=3,
        padding="causal",
        activation="relu"
    )(x)
    x = LayerNormalization()(x)

    x = GlobalAveragePooling1D()(x)

    x = Dense(16, activation="relu")(x)
    x = Dropout(0.3)(x)

    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs, name="tcn_lite")

    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=[
            AUC(name="auc")
        ]
    )

    return model

## Training / Testing Models

### TCN-lite

In [13]:
symbol = "GOOG"
side, _ = await ML_Pipeline(build_tcn_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
63/63 - 2s - 29ms/step - auc: 0.5206 - loss: 0.7353 - val_auc: 0.5017 - val_loss: 0.6678
Epoch 2/20
63/63 - 0s - 3ms/step - auc: 0.5378 - loss: 0.7053 - val_auc: 0.4837 - val_loss: 0.7588
Epoch 3/20
63/63 - 0s - 3ms/step - auc: 0.5197 - loss: 0.6967 - val_auc: 0.4845 - val_loss: 0.7221
Epoch 4/20
63/63 - 0s - 3ms/step - auc: 0.5148 - loss: 0.6992 - val_auc: 0.4754 - val_loss: 0.7106
Epoch 5/20
63/63 - 0s - 3ms/step - auc: 0.5195 - loss: 0.6915 - val_auc: 0.5115 - val_loss: 0.6918
Epoch 6/20
63/63 - 2s - 29ms/step - auc: 0.5344 - loss: 0.6865 - val_auc: 0.5094 - val_loss: 0.6804
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
GOOG: SideSignal.BUY


In [14]:
symbol = "AAPL"
side, _ = await ML_Pipeline(build_tcn_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
63/63 - 3s - 55ms/step - auc: 0.4764 - loss: 0.7247 - val_auc: 0.4988 - val_loss: 0.6784
Epoch 2/20
63/63 - 0s - 3ms/step - auc: 0.5185 - loss: 0.6999 - val_auc: 0.5432 - val_loss: 0.6752
Epoch 3/20
63/63 - 0s - 3ms/step - auc: 0.5087 - loss: 0.6996 - val_auc: 0.4457 - val_loss: 0.6816
Epoch 4/20
63/63 - 0s - 3ms/step - auc: 0.5121 - loss: 0.6963 - val_auc: 0.4135 - val_loss: 0.6883
Epoch 5/20
63/63 - 0s - 3ms/step - auc: 0.5393 - loss: 0.6900 - val_auc: 0.5417 - val_loss: 0.6821
Epoch 6/20
63/63 - 0s - 3ms/step - auc: 0.5291 - loss: 0.6946 - val_auc: 0.5361 - val_loss: 0.6759
Epoch 7/20
63/63 - 0s - 3ms/step - auc: 0.5569 - loss: 0.6869 - val_auc: 0.4684 - val_loss: 0.6824
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
AAPL: SideSignal.BUY


In [ ]:
symbol = "MCFT"
side, _ = await ML_Pipeline(build_tcn_lite, symbol, {})
print(f"{symbol}: {side}")